In [35]:
import os
from scipy.io import loadmat
import pandas as pd
import numpy as np

In [46]:
def load_excel_results(folder_path):
    """
    Load all Excel results from a given folder, extracting metadata from filenames.

    Parameters:
    - folder_path (str): Path to the folder containing Excel files.

    Returns:
    - dict: A dictionary containing DataFrames indexed by (subject, week, session).
    """
    results = {}
    files = os.listdir(folder_path)
    for file in files:
        if file.endswith('.xlsx'):
            # Extract details from the filename
            parts = file.split('__')
            subject_week = parts[0].split('_')
            subject = subject_week[0] + '_' + subject_week[1]
            week = subject_week[2]
            session = parts[1].split('_')[1]

            # Load the Excel file
            file_path = os.path.join(folder_path, file)
            df = pd.read_excel(file_path)

            # Store in dictionary
            if subject not in results:
                results[subject] = {}
            if week not in results[subject]:
                results[subject][week] = {}
            results[subject][week][session] = df

            print(f"Loaded data for {subject}, {week}, {session} from {file_path}")
    return results

# Example usage
folder_path = r'C:\Users\hugma\diss\Breathwork EEG Data\Dreem_Pilot\processed_results_bplzc_tau4'
loaded_excel_results = load_excel_results(folder_path)

Loaded data for s01_week, 1, 1 from C:\Users\hugma\diss\Breathwork EEG Data\Dreem_Pilot\processed_results_bplzc_tau4\s01_week_1__wk1_1__results_tau4.xlsx
Loaded data for s01_week, 1, 2 from C:\Users\hugma\diss\Breathwork EEG Data\Dreem_Pilot\processed_results_bplzc_tau4\s01_week_1__wk1_2__results_tau4.xlsx
Loaded data for s01_week, 1, 3 from C:\Users\hugma\diss\Breathwork EEG Data\Dreem_Pilot\processed_results_bplzc_tau4\s01_week_1__wk1_3__results_tau4.xlsx
Loaded data for s01_week, 1, 4 from C:\Users\hugma\diss\Breathwork EEG Data\Dreem_Pilot\processed_results_bplzc_tau4\s01_week_1__wk1_4__results_tau4.xlsx
Loaded data for s01_week, 1, 5 from C:\Users\hugma\diss\Breathwork EEG Data\Dreem_Pilot\processed_results_bplzc_tau4\s01_week_1__wk1_5__results_tau4.xlsx
Loaded data for s01_week, 1, 6 from C:\Users\hugma\diss\Breathwork EEG Data\Dreem_Pilot\processed_results_bplzc_tau4\s01_week_1__wk1_6__results_tau4.xlsx
Loaded data for s01_week, 1, 7 from C:\Users\hugma\diss\Breathwork EEG Data\

In [47]:
def load_and_adjust_timestamps(filepath):
    """
    Loads and adjusts timestamps from a .mat file.

    Parameters:
    - filepath (str): Path to the .mat file containing timestamps.

    Returns:
    - np.array: Adjusted timestamps as floats.
    """
    data = loadmat(filepath)
    timestamps = data['timestamps'].squeeze()  # Adjust the key if necessary
    adjusted_timestamps = (timestamps - 1) / 1000  # Convert to seconds and adjust if needed
    return adjusted_timestamps

def process_excel_sessions(excel_results, base_timestamp_path):
    """
    Processes Excel sessions by attaching timestamp data from .mat files.
    """
    processed_results = {}

    for subject, weeks in excel_results.items():
        corrected_subject = subject.split('_')[0]  # Assumes subject includes unnecessary suffix like '_week'
        processed_results[corrected_subject] = {}

        for week, sessions in weeks.items():
            week_key = f"week_{week}"  # Formats week key name
            processed_results[corrected_subject][week_key] = {}

            for session, data in sessions.items():
                session_key = f"session_{session}"  # Formats session key name
                print(f"Processing {corrected_subject}, {week_key}, {session_key}")
                session_filename = f"{corrected_subject}_wk{week}_{session}_timestamps.mat"

                timestamp_file = os.path.join(base_timestamp_path, corrected_subject, f"week_{week}", "timestamps", session_filename)
                
                try:
                    adjusted_timestamps = load_and_adjust_timestamps(timestamp_file)
                    data['timestamps'] = adjusted_timestamps
                    processed_results[corrected_subject][week_key][session_key] = data
                except Exception as e:
                    print(f"Failed to load timestamps: {timestamp_file}: {e}")

    return processed_results


# Example usage
base_timestamp_path = "C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot"
processed_excel_data = process_excel_sessions(loaded_excel_results, base_timestamp_path)
                

 

Processing s01, week_1, session_1
Processing s01, week_1, session_2
Processing s01, week_1, session_3
Processing s01, week_1, session_4
Processing s01, week_1, session_5
Processing s01, week_1, session_6
Processing s01, week_1, session_7
Processing s01, week_2, session_1
Processing s01, week_2, session_2
Processing s01, week_2, session_3
Processing s01, week_2, session_4
Processing s01, week_2, session_5
Processing s01, week_2, session_6
Processing s01, week_2, session_7
Processing s01, week_3, session_1
Processing s01, week_3, session_2
Processing s01, week_3, session_3
Processing s01, week_3, session_4
Processing s01, week_3, session_5
Processing s01, week_3, session_6
Processing s01, week_4, session_1
Processing s01, week_4, session_2
Processing s01, week_4, session_3
Processing s01, week_4, session_4
Processing s01, week_4, session_5
Processing s01, week_4, session_6
Processing s01, week_4, session_7
Processing s02, week_1, session_1
Processing s02, week_1, session_2
Processing s02

In [48]:
def save_processed_results(processed_results, save_directory):
    """
    Saves the processed wSMI results stored in a dictionary to CSV files.

    Parameters:
    - processed_results (dict): The dictionary containing the wSMI processing results.
    - save_directory (str): Base directory to save the CSV files.
    """
    for subject, weeks in processed_results.items():
        for week, sessions in weeks.items():
            for session, data in sessions.items():
                if data is not None:
                    # Convert the numpy array or data to a DataFrame
                    df = pd.DataFrame(data)
                    
                    # Ensure the directory exists
                    subject_dir = os.path.join(save_directory, subject, week)
                    if not os.path.exists(subject_dir):
                        os.makedirs(subject_dir, exist_ok=True)

                    # Define the path for saving the file
                    file_path = os.path.join(subject_dir, f'session_{session}_wsmi_tau4.csv')

                    # Save the DataFrame to CSV
                    df.to_csv(file_path, index=False)
                    print(f"Saved processed results to {file_path}")
                else:
                    print(f"No data available to save for {subject} {week} {session}")

# Example usage
base_dir = r'C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot/Processed_Results_bplz_tau4'

save_processed_results(processed_excel_data, base_dir)

Saved processed results to C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot/Processed_Results_bplz_tau4\s01\week_1\session_session_1_wsmi_tau4.csv
Saved processed results to C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot/Processed_Results_bplz_tau4\s01\week_1\session_session_2_wsmi_tau4.csv
Saved processed results to C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot/Processed_Results_bplz_tau4\s01\week_1\session_session_3_wsmi_tau4.csv
Saved processed results to C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot/Processed_Results_bplz_tau4\s01\week_1\session_session_4_wsmi_tau4.csv
Saved processed results to C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot/Processed_Results_bplz_tau4\s01\week_1\session_session_5_wsmi_tau4.csv
Saved processed results to C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot/Processed_Results_bplz_tau4\s01\week_1\session_session_6_wsmi_tau4.csv
Saved processed results to C:/Users/hugma/diss/Breathwork EEG Data/Dreem_Pilot/Processed_Results_bpl